<a href="https://colab.research.google.com/github/osiris/test/blob/develop/ia-nlp-01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctico 1: Preprocesamiento

# 0 - Setup

In [ ]:
! pip install nltk scikit-learn gensim transformers

Descarga de un dataset de detección de Spam

In [ ]:
! wget https://archive.ics.uci.edu/static/public/228/sms+spam+collection.zip
! unzip sms+spam+collection.zip

--2024-09-02 02:18:52--  https://archive.ics.uci.edu/static/public/228/sms+spam+collection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘sms+spam+collection.zip’

sms+spam+collection     [  <=>               ] 198.65K   758KB/s    in 0.3s    

2024-09-02 02:18:53 (758 KB/s) - ‘sms+spam+collection.zip’ saved [203415]

Archive:  sms+spam+collection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


# 1 - NLTK

[*Natural Language Toolkit* (NLTK)](https://www.nltk.org/) es una popular herramienta para hacer procesamiento y preprocesamiento de lenguaje natural.

In [ ]:
import nltk
nltk.download('punkt')  # <-- Signos de puntuacion
nltk.download('stopwords')  # <-- Stopwords en inglés
nltk.download('wordnet')  # <-- WordNet, un extenso vocabulario en inglés
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.lm import Vocabulary
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Probamos con una oración de ejemplo

In [ ]:
# Oración de ejemplo
text = "This is an example sentence. It contains some stop words and punctuation marks!"

Tokenizamos la oración usando el vocabulario provisto por [WordNet](https://wordnet.princeton.edu/).

In [ ]:
# Tokenización
tokens = word_tokenize(text)

print('Texto original:', text)
print('Texto tokenizado:', tokens)

Texto original: This is an example sentence. It contains some stop words and punctuation marks!
Texto tokenizado: ['This', 'is', 'an', 'example', 'sentence', '.', 'It', 'contains', 'some', 'stop', 'words', 'and', 'punctuation', 'marks', '!']


Eliminamos las stopwords.

In [ ]:
# Eliminar stopwords
stop_words = set(stopwords.words('english'))
filtered_tokens = [word for word in tokens if word.lower() not in stop_words]

print('Tokens iniciales:', tokens)
print('Tokens sin stopwords:', filtered_tokens)

Tokens iniciales: ['This', 'is', 'an', 'example', 'sentence', '.', 'It', 'contains', 'some', 'stop', 'words', 'and', 'punctuation', 'marks', '!']
Tokens sin stopwords: ['example', 'sentence', '.', 'contains', 'stop', 'words', 'punctuation', 'marks', '!']


In [ ]:
FUNCTION = 'lemmatization' #@param ["lemmatization", "stemming"]

if FUNCTION == 'lemmatization':
  # Lemmatization
  lemmatizer = WordNetLemmatizer()
  final_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]
elif FUNCTION == "stemming":
  # Stemming
  stemmer = PorterStemmer()
  final_tokens = [stemmer.stem(word) for word in filtered_tokens]

print("Tokens iniciales: ", filtered_tokens)
print(f"Tokens con {FUNCTION}: ", final_tokens)

Tokens iniciales:  ['example', 'sentence', '.', 'contains', 'stop', 'words', 'punctuation', 'marks', '!']
Tokens con lemmatization:  ['example', 'sentence', '.', 'contains', 'stop', 'word', 'punctuation', 'mark', '!']


Armo un vocabulario con los tokens que encontré

In [ ]:
vocab = Vocabulary(final_tokens, unk_cutoff=1)

Puedo ver qué está en el vocabulario y qué no:

In [ ]:
WORD = "aliens" #@param "example"

print("")
print(f"{WORD} --> {vocab.lookup(WORD)}")


aliens --> <UNK>


Podemos empaquetar todo en una clase que nos proveerá de preprocesamiento usando [Scikit-Learn](https://scikit-learn.org/stable/).

In [ ]:
from sklearn.base import TransformerMixin

class PreprocessingEn(TransformerMixin):
  def __init__(self, function):
    self.tokenizer = word_tokenize
    self.function = function

  def fit(self, X=None, y=None):
    if self.function == 'lemmatization':
      # Lemmatization
      self.normalization = WordNetLemmatizer()
      self.norm_fn = self.normalization.lemmatize
    elif self.function == "stemming":
      # Stemming
      self.normalization = PorterStemmer()
      self.norm_fn = self.normalization.stem

    self.stop_words = set(stopwords.words('english'))
    return self

  def transform(self, X, y=None):
    result = []
    for sent in X:
      tokens = self.tokenizer(sent)
      filtered_tokens = [word for word in tokens if word.lower() not in self.stop_words]
      final_tokens = [self.norm_fn(word) for word in filtered_tokens]
      result.append(final_tokens)
    return result

In [ ]:
corpus = [
    "This is a sample sentence.",
    "Another example sentence.",
    "The kid on top of a truck.",
]

preprocess = PreprocessingEn('lemmatization').fit()

result = preprocess.transform(corpus)

for original, preprocessed in zip(corpus, result):
  print('Original: ', original)
  print('Preprocesada: ', preprocessed)
  print('\n')

Original:  This is a sample sentence.
Preprocesada:  ['sample', 'sentence', '.']


Original:  Another example sentence.
Preprocesada:  ['Another', 'example', 'sentence', '.']


Original:  The kid on top of a truck.
Preprocesada:  ['kid', 'top', 'truck', '.']




# 2 - No hay vocabulario inicial

In [ ]:
! curl --header 'Host: drive.usercontent.google.com' --header 'User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8' --header 'Accept-Language: en-US,en;q=0.6' --header 'Cookie: SID=bgiZWLaLYKq4iqj3nXroMprEr9bvkh4ih-bfBLqwsTuH3i8H5l9Tg3YqsBy6M_cm0bBLBQ.; __Secure-1PSID=bgiZWLaLYKq4iqj3nXroMprEr9bvkh4ih-bfBLqwsTuH3i8HK67gdLIU9g1faDiz5j6eMg.; __Secure-3PSID=bgiZWLaLYKq4iqj3nXroMprEr9bvkh4ih-bfBLqwsTuH3i8H-lF0WKlXjGb5_8n1Tg1MXA.; HSID=Ar439EY6o482Taaxh; SSID=A4ayVEK6mAxKuBIgM; APISID=yfFe90xPRuPmP93J/AVFZaXfhhSQnvLgCl; SAPISID=GC_yfKKqBi0J9jk5/AmhdIWSRCQ1SZkwmw; __Secure-1PAPISID=GC_yfKKqBi0J9jk5/AmhdIWSRCQ1SZkwmw; __Secure-3PAPISID=GC_yfKKqBi0J9jk5/AmhdIWSRCQ1SZkwmw; SEARCH_SAMESITE=CgQIrpkB; AEC=Ackid1QuAfCf-cQ9GfyuIqY08n-9hZCCEyxP2aPefjiwgRzi1Zyvwhao7Bs; 1P_JAR=2023-09-30-00; OGPC=19037325-1:; NID=511=LOGKeYp-8pcIp4RzbyKF4MUvptab14CMrJ0cAjuWh1yjkUb_-hHEgH8JMgE6QUuDn2G0fJ5cDSEk7cgzlgU6RSPNsdyhpLEsYLY9l1T4KsqLKu2SKuDqnJWL17lY04mIpJgnebAUhOnFp-ceQMwNxuhL7ftgf2li5xWGw-JeLgjP4GzhtiqLbv06cmSjN8sX_NufLsZGDMQ54aJKPORtCmpqeTrC-bM9oJkf-zptychUlEN-x359JIZD6yhhVTak1QZpe06frNTEbYV5W55x_rj4CakShH_mJbHeJzuS3IerqDnItrr4REEYbyP3zCms0LqqY9yUHEkSjk44pLweB4kigFWnZfXUN7gvWvLJrvjT26OvBuuyM62poTQf7xILTPIpEiwUZIHTJkOxLVj0tU28SrSLpZhrrBe_dk3L49YI8b5Qtw; __Secure-1PSIDTS=sidts-CjIB3e41hdD7BRKfsNeNGefreYcgjwkivhXbvnkNuCLQEfDp31IbK_NyYY4RqwMJsq-MNBAA; __Secure-3PSIDTS=sidts-CjIB3e41hdD7BRKfsNeNGefreYcgjwkivhXbvnkNuCLQEfDp31IbK_NyYY4RqwMJsq-MNBAA; SIDCC=ACA-OxMjzikCAYLCiKKi-6vbf3O4agWE1kO27kcb0ZtIV9A56gtkF0RsUn01gT_4HAWsSRoXE3s; __Secure-1PSIDCC=ACA-OxMVtBywb2tRgiNKkNAzjDt5hZBjPuNvGR65wqP9LEOpYBTdkNkqxGI9zBNhbL6cDspQJDw; __Secure-3PSIDCC=ACA-OxPRNpyuA_5drqWExqOyTgIO9abchysVwFjkNpP7PvPMx1-jnzee7Sbq3_jR1neUPrhkuXd7' --header 'Connection: keep-alive' 'https://drive.usercontent.google.com/download?id=1VchoJFlPCeFfG5-aRnVIe2UGVA3b4ieQ&export=download&authuser=1&confirm=t&uuid=456f5ebb-6b69-44c0-adf9-d28b4cf354bb&at=APZUnTU-FH5rnS9pw3lddqLA0I92:1696181034208' -L -o 'martin_fierro.txt'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 67632  100 67632    0     0  27971      0  0:00:02  0:00:02 --:--:-- 27981


In [ ]:
with open('martin_fierro.txt', encoding="utf-8") as fl:
  martin_fierro_raw = fl.read()

In [ ]:
print(martin_fierro_raw[25000:25300])

a indagación:
si había venido al cantón
en tal tiempo o en tal otro...
y si había venido en potro,
en reyuno o redomón.

131
Y todo era alborotar
al ñudo, y hacer papel;
conocí que era pastel
pa engordar con mi guayaca;
mas si voy al coronel
me hacen bramar en la estaca.

132
¡Ah, hijos de una...! ¡


Podemos ajustar un vocabulario y crear nuestra clase de preprocesamiento

In [ ]:
class PreprocessFierro(TransformerMixin):
  def __init__(self, unk_cutoff=2):
    self.unk_cutoff = unk_cutoff

  def fit(self, X=None, y=None):
    words = X.lower().split()  # <-- Pasar a minúscula y separar por espacios en blanco
    self.vocab = Vocabulary(words, unk_cutoff=self.unk_cutoff)

  def transform(self, X, y=None):
    result = []
    for sent in X:
      tokens = [self.vocab.lookup(x) for x in sent.lower().split()]
      result.append(tokens)
    return result


In [ ]:
corpus = [
    "Una oración en español.",
    "si había venido al cantón en tal tiempo o en tal otro...",
    "A sentence out of context."
]

preprocess = PreprocessFierro(unk_cutoff=10)
preprocess.fit(martin_fierro_raw)
result = preprocess.transform(corpus)

for original, preprocessed in zip(corpus, result):
  print('Original: ', original)
  print('Preprocesada: ', preprocessed)
  print('\n')

Original:  Una oración en español.
Preprocesada:  ['una', '<UNK>', 'en', '<UNK>']


Original:  si había venido al cantón en tal tiempo o en tal otro...
Preprocesada:  ['si', 'había', '<UNK>', 'al', '<UNK>', 'en', 'tal', '<UNK>', 'o', 'en', 'tal', '<UNK>']


Original:  A sentence out of context.
Preprocesada:  ['a', '<UNK>', '<UNK>', '<UNK>', '<UNK>']




## Expresiones regulares

También conocido popularmente como **RegEx**. Es un mini lenguaje de programación diseñado para realizar búsquedas en strings. Son extremadamente útiles para:
- Extraer datos de distintos tipos de archivos, texto o con otro tipo de codificación.
- Web scraping: como veremos en las próximas clases, las regex son un buen método para encontrar la información que se necesita en un sitio web.
- Limpieza de datos: herramienta fundamental en el repertorio del científico de datos para limpiar datos quitando caracteres "ruidosos", o armando nuevos "features" según la presencia o no de cierto texto.


fuente: [Humai - NLP 1](https://github.com/institutohumai/cursos-python/tree/master/NLP/1_Introduccion)

In [ ]:
doc = 'Usualmente existe una relación costo beneficio entre las distintas técnicas'

Recursos útiles para regex:

- [Sitio para armar RegEx online](https://regexr.com/)
- [Alternativa](https://regex101.com/)
- [CheatSheet](https://www.dataquest.io/wp-content/uploads/2019/03/python-regular-expressions-cheat-sheet.pdf)


Python utiliza la libreria llamada **re** para todo lo relacionado a regular expressions

In [ ]:
# Tokenización simple con RegEx
import re
re.findall('\w+', doc)

['Usualmente',
 'existe',
 'una',
 'relación',
 'costo',
 'beneficio',
 'entre',
 'las',
 'distintas',
 'técnicas']

In [ ]:
import re

# a- extraer números de una oración.
texto = "Mi nombre es Juan y mi teléfono es 1564232324"
regla_de_busqueda = "\d+"
# print(texto)
print(re.findall(regla_de_busqueda, texto))

['d']


Las funciones principales de la librería re son:
- re.findall(pattern, string) para encontrar todos los resultados de una búsqueda
- re.search(pattern, string) para encontrar el primer resultado que coincida
- re.sub(pattern, replace, string) para substituir un texto por otro


<h2><center>Sintaxis para construir regex</center></h2>


<h3><center>Grupos de captura</center></h3>


|     |                       |
|-----|-----------------------|
| ()  | grupo de captura      |
|(?:) | grupo de no captura   |

<h3><center>Operadores</center></h3>

|         |                      |
|---------|----------------------|
| \|      | operador "or"        |
| \\      | Escapar, o interpretar literalmente |
| []      | conjunto (cada elemento estará automáticamente separado por "or")             |
|[m-z3-9] | rangos               |


<h3><center>Cuantificadores</center></h3>

|      |                                              |
|------|----------------------------------------------|
| +    | Uno o más del elemento anterior              |
| *    | Cero o más del elemento anterior             |
| {4,} | Cuatro o más del elemento anterior           |
| ?    | Cambia el operador anterior de lazy a greedy |

#### ¿Cómo se usa? Veamos ejemplos

In [ ]:
# En realidad los telefonos no son cualquier seguidilla de numeros
# suelen tener entre 6 y 8 numeros despues del 15
texto = "Mi nombre es María y mi teléfono es 1564232324"
regla_de_busqueda = "15\d{6,8}"
re.findall(regla_de_busqueda,texto)

['1564232324']

In [ ]:
# En realidad los telefonos no arrancan siempre con 15
# capaz empiezan con 11 si son de buenos aires por ejemplo
texto = "Mi nombre es Carlos y mi teléfono es 114232324 154232324"
regla_de_busqueda = "(?:15|11)\d{6,8}"
re.findall(regla_de_busqueda,texto)

['114232324', '154232324']

In [ ]:
# En realidad los telefonos pueden tener un guión o espacio a parte de números
texto = "Mi nombre es asfasfeaf33 y mi teléfono es 11 6423-2324"
regla_de_busqueda = "(?:15|11)[0-9\s-]{6,10}"
re.findall(regla_de_busqueda,texto)

['11 6423-2324']

In [ ]:
# Ejercicio: extraer todos los mails

texto = "Hola te paso mi mail python@hotmail.com, saludos. Si no te funciona mandame a este otro, pedro_2010@yahoo.com"


## Distancia de edición

Para afrontar el "ruido" en nuestros textos, tenemos una importante herramienta: la distancia de edición.

También llamada Distancia de Levenshtein, nos dice la cantidad mínima de operaciones requerida para llevar de un string al otro. Expresandola de una manera sencilla (y en su versión sin normalizar entre 0 y 1) podemos explicarla así:

$$\mathit{L} = S+D+I$$

Donde $S$ = sustitución, $D$ = Eliminación, $I$ = inserción (de un caracter).


Como en muchos idiomas la raíz importa más que el resto de la palabra, a veces se usa la métrica Jaro Winkler que pondera más los caracteres iniciales.

In [ ]:
%%capture
!pip install pyjarowinkler
from nltk.metrics import edit_distance
from pyjarowinkler import distance as jwdist

In [ ]:
# Definimos una lista de tuplas de palabras
palabras = [("pero", "perro"),("pero", "pierdo"), ("nueve", "mueve"),  ("totalmente","diferentes"), ("pero", "basta")]

# Calculamos las metricas de distancia pasando cada tupla como argumentos a levdist() y get_jaro_distance()
for x,y in palabras:
    print(f"'{x}' vs '{y}':")
    print("Distancia Levenshtein ->", edit_distance(x,y))
    print("Similitud Jaro Winkler ->",jwdist.get_jaro_distance(x,y))
    print("-"*40)

'pero' vs 'perro':
Distancia Levenshtein -> 1
Similitud Jaro Winkler -> 0.95
----------------------------------------
'pero' vs 'pierdo':
Distancia Levenshtein -> 2
Similitud Jaro Winkler -> 0.9
----------------------------------------
'nueve' vs 'mueve':
Distancia Levenshtein -> 1
Similitud Jaro Winkler -> 0.87
----------------------------------------
'totalmente' vs 'diferentes':
Distancia Levenshtein -> 7
Similitud Jaro Winkler -> 0.52
----------------------------------------
'pero' vs 'basta':
Distancia Levenshtein -> 5
Similitud Jaro Winkler -> 0.0
----------------------------------------


## Búsqueda difusa de strings (fuzzy string matching)

Técnica muy útil para tener en el repertorio, nos permite encontrar coincidencias que no son exactas. En el caso más sencillo, la búsqueda difusa nos devolverá un puntaje de similitud entre los strings relacionada a las diferencias entre los caracteres. Existen variaciones que consideran cuántas unidades (palabras, tokens) coinciden, que ordenan la oración antes, o que borran repetidos.

In [ ]:
%%capture
!pip install thefuzz[speedup]
from thefuzz import process, fuzz

In [ ]:
def get_match(string, lista_strings):
    """Para buscar coincidencias entre nombre con los nombres"""
    mejor_match, puntaje = process.extractBests(string.strip(), lista_strings, scorer=fuzz.token_set_ratio)[0]
    return mejor_match, puntaje

In [ ]:
# Completar
busqueda = 'Pebro'
candidatos = 'María, Fabio, Paula, Natu, Pedro, Miguel'.split(', ')

get_match(busqueda, candidatos)

('Pedro', 80)

# SpaCy

Mientras NLTK se centra en técnicas más bien tradicionales, [spaCy](https://spacy.io/) ofrece una API muy cómoda que integra modelos de aprendizaje automático que resuelven tareas típicas de lingüística computacional.


In [ ]:
%%capture
!python -m spacy download es_core_news_md
import numpy as np
import spacy
from spacy import displacy

In [ ]:
# Inicializamos Spacy con modelos en español
nlp = spacy.load('es_core_news_md')

In [ ]:
ejemplo = 'El Doctor del Hospital Austríaco le dijo a Josefina que actuando de maneras probabilísticas el cerebro aprende conceptos discretos. ¡Qué fenómeno misterioso!'

In [ ]:
# Instanciamos un Doc de spacy con nuestro texto
doc = nlp(ejemplo)
type(doc)

spacy.tokens.doc.Doc

In [ ]:
for word in doc:
    print(word.text)

El
Doctor
del
Hospital
Austríaco
le
dijo
a
Josefina
que
actuando
de
maneras
probabilísticas
el
cerebro
aprende
conceptos
discretos
.
¡
Qué
fenómeno
misterioso
!


In [ ]:
doc

El Doctor del Hospital Austríaco le dijo a Josefina que actuando de maneras probabilísticas el cerebro aprende conceptos discretos. ¡Qué fenómeno misterioso!

## Lemmatización

El *stemming* vemos que es un enfoque bastante bruto para normalizar, desechando parte de la información de la palabra. Podemos tener un problema de polisemia, con una raíz refiriendo a palabras muy distintas. La solución a esto es la **lematización**, que busca llevar a una palabra a su forma canónica o esencial.

In [ ]:
for word in doc:
    print(word.lemma_)

el
Doctor
del
Hospital
Austríaco
él
decir
a
Josefina
que
actuar
de
manera
probabilística
el
cerebro
aprender
concepto
discreto
.
¡
qué
fenómeno
misterioso
!


## Limitaciones de la Bolsa de Palabras

Utilizando las técnicas que vimos (normalización, n-gramas, stop words) podemos mejorar significativamente el rendimiento de algoritmos de Aprendizaje Automático que se basan en bolsa de palabras.

Sin embargo, si entrenamos un clasificador usando esta matriz, lo que hará es encontrar relaciones estadísticas entre la ocurrencia de cierto token con su categoría (en el caso supervisado). ¿Qué problemas se te puede que puede tener?

- Contexto
- Orden
- Dimensionalidad (variaciones en las palabras, n gramas)


En las próximas clases veremos cómo las redes neuronales abordan estos obstáculos.

# Tareas de NLP

SpaCy integra en su API de una manera elegante atributos y métodos generados por modelos de Aprendizaje Automático.

## Segmentación de oraciones

In [ ]:
list(doc.sents)

[El Doctor del Hospital Austríaco le dijo a Josefina que actuando de maneras probabilísticas el cerebro aprende conceptos discretos.,
 ¡Qué fenómeno misterioso!]

## Part-Of-Speech Tags

Podemos estimar la función gramatical de cada palabra, accediendo directamente al atributo `.pos_` de cada token:


In [ ]:
for word in doc:
    print(word.text, word.pos_)

El DET
Doctor PROPN
del ADP
Hospital PROPN
Austríaco PROPN
le PRON
dijo VERB
a ADP
Josefina PROPN
que SCONJ
actuando VERB
de ADP
maneras NOUN
probabilísticas ADJ
el DET
cerebro NOUN
aprende VERB
conceptos NOUN
discretos ADJ
. PUNCT
¡ PUNCT
Qué DET
fenómeno NOUN
misterioso ADJ
! PUNCT


## Dependency parsing

¡También contamos con información sobre la co-referencia o dependencia sintáctica entre términos!

In [ ]:
displacy.render(doc, style='dep', jupyter=True)

## Reconocimiento de Entidades

Podemos encontrar las personas (más general, "entes") que se mencionan en los textos:

In [ ]:
doc.ents

(Doctor, Hospital Austríaco, Josefina)

In [ ]:
displacy.render(doc, style='ent', jupyter=True);